In [ ]:
from pathlib import Path
import pyemu
import pandas as pd

import pypestvis as ppv

## Load control file and clean up observation metadata
* fill in k,i,j from idx0 where needed
* fixup kper, kstp
* define `slider` column

In [ ]:
rd = "." # "batch_20250801"
md = Path(rd, "lheg_ies")
# load history matching pst
pst = pyemu.Pst(Path(md, 'lhgzsi.pst').as_posix())
obs = pst.observation_data
scenmap = pd.read_csv(Path(md, "scenario.csv")).set_index('kper')
chdmap = scenmap.CHD.to_dict()

# fix up i,j,ks to make zero based
# obs.groupby('obgnme').first()
# clean up metadata
# fill kper etc
obs[['kper', 'kstp']] = obs[['kper', 'kstp']].astype('Int32').fillna(0)
# fixing issue with dummy obs that made it incompatible
obs['idx0'] = obs.idx0.replace('dummy', None)

# in this instance safe to fill k,i,j with idx0...
obs = obs.fillna({'k': obs.idx0, 'i': obs.idx1, 'j': obs.idx2}).astype(
    {c: "Int32" for c in ['k', 'i', 'j', 'kstp', 'kper']}).fillna({'k': 0})

# need annoying one-based (parfile tables) to zerobased
obs.loc[obs.obgnme.str.contains("chd|ghb", na=False), ['k', 'i', 'j']] -= 1
obs['slider'] = obs.kper.apply(lambda x: (x, chdmap[int(x)]))
# put back on pest object
pst.observation_data = obs

## Build Handler passing `slider` as column to use for slider widget (although not actually used in this e.g.).

In [ ]:
vh = ppv.VisHandler(pst, wd=Path(md), tidx='slider')

## Display:

In [ ]:
display(vh.default_map_layout)
display(vh.default_unmap_layout)